In [1]:
import sys

sys.path.insert(0, "../../")

In [6]:
from selenium.webdriver.common.by import By
import datacrew.crawler.crawler as dcc
import datacrew.crawler.article as dca

BASE_URL = "https://domo-support.domo.com"

IMG_BASE_URL = "https://domo-support.domo.com/servlet/rtaImage"

OUTPUT_FOLDER = "../../raw_kb"

driver = dcc.driversetup(is_headless=False)

HOME_URL = "https://domo-support.domo.com/s/knowledge-base"

In [7]:
article = dca.Article_KB_Home(
    url=HOME_URL,
    base_url=BASE_URL,
    driver=driver)


💤 retrieving https://domo-support.domo.com/s/knowledge-base 💤
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC, 0TO5w000000ZamsGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC, 0TO5w000000ZammGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamzGAC, 0TO5w000000ZamzGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC, 0TO5w000000ZamoGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamnGAC, 0TO5w000000ZamnGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamlGAC, 0TO5w000000ZamlGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamqGAC, 0TO5w000000ZamqGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000Zan0GAC, 0TO5w000000Zan0GAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000Zan2GAC, 0TO5w000000Zan2GAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZampGAC, 0TO5w000000ZampGAC
❤️ child url - htt

In [12]:
article

Article_KB_Home(url='https://domo-support.domo.com/s/knowledge-base', base_url='https://domo-support.domo.com', entity_base_url='https://domo-support.domo.com/s/knowledge-base', url_entity_prefix='/s/knowledge-base', url_id='', is_success=True, url_ls=['https://domo-support.domo.com/s/knowledge-base', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamzGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamnGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamlGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamqGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000Zan0GAC', 'https://domo-support.domo.com/s/topic/0TO5w000000Zan2GAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZampGAC', 'https://domo-support.domo.com/s/topi

In [14]:
import re

def convert_to_snake(text_str):
    """converts 'snake_case_str' to 'snakeCaseStr'"""

    return text_str.replace(" ", "_").lower()


def clean_url_name(path_name):
    valid_chars = r"[^a-zA-Z0-9_]"

    return re.sub(valid_chars, "", path_name)


def get_id_from_url(url: str, url_match: str):
    """use url_match" to identify the id of an object"""
    return url.split(url_match)[1].split("/")[0]

In [16]:
import requests
import pandas as pd


def download_img(image_url, image_path, debug_prn: bool = False):

    img_data = requests.get(image_url).content

    with open(image_path, "wb") as handler:
        if debug_prn:
            print(f"downloading {image_url} to {image_path}")
        handler.write(img_data)


def download_article_images(
    article: dca.Article,
    output_path: str,
    debug_prn: bool = False,
):
    image_ls = article.image_ls

    if not image_ls:
        return 

    # download images
    for img in image_ls:
        img_name = img.get("name")

        if not img_name:
            continue

        img_url = img.get("url")
        img_path = f"{output_path}/{img_name}"
        img_rel_path = img.get("relative_url")

        if debug_prn:
            print(
                f"downloading {img_url} to {img_path}.  replacing article with {img_rel_path} with {img_name}"
            )

        download_img(image_url=img_url, image_path=img_path)

        article.md_str = article.md_str.replace(img_rel_path, img_name)

    return image_ls

In [17]:
from mdutils.mdutils import MdUtils
import pandas as pd

import os


def create_output_folder(output_folder, title: str):
    output_path = os.path.join(
        output_folder, clean_url_name(convert_to_snake(title))
    )

    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path


def output_html(output_html_path, soup):
    with open(output_html_path, 'w') as f:
        f.write(str(soup))


def process_category(article: dca.Article,
                     output_folder: str,
                     debug_prn: bool = False):

    output_path = create_output_folder(os.path.join( output_folder, 'home'), article.category) 
    
    output_html_path = os.path.join(output_path, 'index.html')
    
    output_html(output_html_path, article.soup)

    download_article_images(
        article=article,
        output_path=output_path)


process_category(article, output_folder=OUTPUT_FOLDER)
